In [5]:
import os
import re 
import rmgpy.chemkin

Test with a regular chemkin

Make the folders

In [6]:
blends_directory = '/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/concatenate_equal_parts/'

list_of_blends = [blend for blend in os.listdir(blends_directory) if re.match('([A-Z0-9]+_[A-Z0-9]+)', blend)]

In [23]:
' '.join(list_of_blends)

'C2H5F_CH2F2 C2H5F_CH2FCH2F C2H5F_CH2FCHF2 C2H5F_CH3CHF2 C2H5F_CH3F CH2F2_CH3CF3 CH2F2_CH3F CH2FCH2F_CH2F2 CH2FCH2F_CH2FCHF2 CH2FCH2F_CH3CHF2 CH2FCH2F_CH3F CH2FCHF2_CH2F2 CH2FCHF2_CH3F CH3CF3_C2H5F CH3CF3_CH2FCH2F CH3CF3_CH2FCHF2 CH3CF3_CH3CHF2 CH3CHF2_CH2F2 CH3CHF2_CH2FCHF2 CH3F_CH3CF3 CH3F_CH3CHF2'

In [24]:
len(list_of_blends)

21

In [5]:
for blend in list_of_blends: 
    os.mkdir(f'/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts_species_in_core_redo/{blend}')

Gather the species to be added into the core

In [9]:
def load_chemkin_file(path): 
    """ 
    Load Chemkin file and extract the reactions and species.
    Path should be the path to the species folder.
    """
    
    full_path = os.path.join(path,'concatenated')
    chemkin_path = os.path.join(full_path, 'fixed_chem.inp')
    dictionary_path = os.path.join(full_path,'species_dictionary.txt')
    transport_path = os.path.join(full_path,'tran.dat')


    species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_path, dictionary_path=dictionary_path, transport_path=transport_path)

    return species_list, reaction_list 



def write_rmg_input_file(species_dictionary, spc1, spc2):

    input_file = ''' 

thermolibs = [
'primaryThermoLibrary',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'Fluorine',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]

thermolibs_Creg = [
'primaryThermoLibrary',
'FFCM1(-)',
'DFT_QCI_thermo',
]



database(
thermoLibraries = thermolibs,
reactionLibraries = ['FFCM1(-)','halogens_pdep'],
seedMechanisms = ['FFCM1(-)'],
kineticsDepositories = ['training'],
kineticsFamilies = ['default','halogens','Disproportionation-Y'],
frequenciesLibraries = ['halogens_G4'],
kineticsEstimator = 'rate rules',
)


    
species(
    label = 'N2',
    reactive = False,
    structure = SMILES('N#N')
)
    
    
    
'''
    
    for key in species_dictionary.keys(): 
        print(f'adding in species: {key} ')    
        
        block = f"""

species(
    label = '{key}',
    reactive = True,
    structure = SMILES('{species_dictionary[key]}')
)      
        
        """
        input_file += block
        
        
    block = '''
    
simulator(
    atol = 1e-16,
    rtol = 1e-08,
    sens_atol = 1e-06,
    sens_rtol = 0.0001,
)


generatedSpeciesConstraints(
    allowed=['input species','seed mechanisms','reaction libraries'],
    maximumCarbonAtoms=8,
    maximumOxygenAtoms=6,
    #maximumHeavyAtoms=24,
    maximumRadicalElectrons=2,
    maximumSingletCarbenes=1,
    maximumCarbeneRadicals=0,
    allowSingletO2 = True,
)

options(
    units = "si",
    generateSeedEachIteration = True,
    generateOutputHTML = True,
    generatePlots = True,
    saveSimulationProfiles = True,
    saveEdgeSpecies = False,
    keepIrreversible = True,
    verboseComments = False,
)
    
    
simpleReactor(
        temperature=[(1000,'K'),(2000,'K')],
        pressure= [(1.0,'bar'),(10.0,'bar')],
        nSims=10,
        initialMoleFractions={
'''

    input_file += block
    
    block = \
f'''
        "{spc1}": 0.5,
        "{spc2}": 0.5,
        "O2": 1,
        "N2": 3.76,
        
'''
    input_file += block
    
    block = \
'''
        },
        # terminationConversion={
        # 'halogen': 0.999,
        # },
        #terminationRateRatio=1e-4,
        #terminationTime=(10,'s'),
        terminationTime=(1,'s'),
        #sensitivity=['halogen','OH'],
        #sensitivityThreshold=0.001,
        )
        
model(
    toleranceMoveToCore = 0.1,
    toleranceInterruptSimulation = 0.1,
    maximumEdgeSpecies = 3e5,
    filterReactions = True,
    filterThreshold = 5e8,
    minCoreSizeForPrune = 50,
    minSpeciesExistIterationsForPrune = 4,
)

pressureDependence(
    method='modified strong collision',
    maximumGrainSize=(0.5,'kcal/mol'),
    minimumNumberOfGrains=250,
    temperatures=(300,2500,'K',8),
    pressures=(0.01,100,'bar',5),
    interpolation=('Chebyshev', 6, 4),
    maximumAtoms=16,
)

    
    
    '''
    input_file += block
    
    
    return input_file

In [31]:
list_ = []
for blend in list_of_blends:
    if not os.path.exists(f'/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts_species_in_core_redo/{blend}/chemkin'):
        list_.append(blend)
    
    
    
    
    
    

In [32]:
' '.join(list_)

'C2H5F_CH2F2 C2H5F_CH2FCH2F C2H5F_CH2FCHF2 C2H5F_CH3CHF2 C2H5F_CH3F CH3CF3_C2H5F'

In [10]:
#get mapping

#dictionary with composition of each species 

compositions = {'C2H5F':{'C': 2,'H': 5,'F': 1},
                 'C2H6':{'C': 2,'H': 6},
                 'CH2F2': {'C':1,'H':2,'F':2},
                 'CH2FCH2F':{'C':2,'H':4,'F':2},
                 'CH2FCHF2': {'C':2,'H':3,'F':3},
                 'CH3CF3': {'C':2,'H':3,'F':3},
                 'CH3CHF2': {'C':2,'H':4,'F':2},
                 'CH3F':{'C':1,'H':3,'F':1},
                 'CH4': {'C':1,'H':4},
}

smiles = {'C2H5F':'CCF',
         'C2H6':'CC',
         'CH2F2': 'FCF',
         'CH2FCH2F':'FCCF',
         'CH2FCHF2': 'FCC(F)F',
         'CH3CF3': 'CC(F)(F)F',
         'CH3CHF2': 'CC(F)F',
         'CH3F': 'CF',
         'CH4': 'C',
          'O2': '[O][O]',
}


#get the species ID's of each of the species in the blends
#let's save this to a dictionary that we can reference in functions below

mapping = dict()


for blend in list_of_blends: 
    
    try:

        match = re.findall('([A-Z0-9]+)_([A-Z0-9]+)',blend)
        spec1 = match[0][0]
        spec2 = match[0][1]

        if len(match)>2:
            print(f'More than two species found: {blend}')

        direc = f'/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/concatenate_equal_parts/{blend}/concatenated/'
        spcs, rxns = rmgpy.chemkin.load_chemkin_file(f'{direc}fixed_chem.inp',f'{direc}species_dictionary.txt', f'{direc}tran.dat' )
        
        for_this_blend=[]

        for species in spcs: 
            if species.smiles == smiles[spec1]: 
                for_this_blend.append(species.to_chemkin())
            if species.smiles == smiles[spec2]: 
                for_this_blend.append(species.to_chemkin())

        #did this in a separate list because I want to make sure O2 is last in the list
        for species in spcs: 
            if species.smiles == smiles['O2']: 
                for_this_blend.append(species.to_chemkin())

 
        #add to dictionary
        mapping[blend] = for_this_blend

    except: 
    
        print(blend)
        #print(e)

#print(mapping)


print(len(mapping.keys()), len(list_of_blends))





21 21


In [11]:
print(mapping)

{'C2H5F_CH2F2': ['C2H5F(1)', 'CH2F2(35)', 'O2(2)'], 'C2H5F_CH2FCH2F': ['C2H5F(1)', 'S(179)', 'O2(2)'], 'C2H5F_CH2FCHF2': ['C2H5F(1)', 'S(179)', 'O2(2)'], 'C2H5F_CH3CHF2': ['C2H5F(1)', 'S(179)', 'O2(2)'], 'C2H5F_CH3F': ['C2H5F(1)', 'CH3F(33)', 'O2(2)'], 'CH2F2_CH3CF3': ['CH2F2(1)', 'CH3CF3(86)', 'O2(2)'], 'CH2F2_CH3F': ['CH2F2(1)', 'CH3F(21)', 'O2(2)'], 'CH2FCH2F_CH2F2': ['C2H4F2(1)', 'CH2F2(39)', 'O2(2)'], 'CH2FCH2F_CH2FCHF2': ['C2H4F2(1)', 'S(145)', 'O2(2)'], 'CH2FCH2F_CH3CHF2': ['C2H4F2(1)', 'S(145)', 'O2(2)'], 'CH2FCH2F_CH3F': ['C2H4F2(1)', 'CH3F(37)', 'O2(2)'], 'CH2FCHF2_CH2F2': ['C2H3F3(1)', 'CH2F2(38)', 'O2(2)'], 'CH2FCHF2_CH3F': ['C2H3F3(1)', 'CH3F(139)', 'O2(2)'], 'CH3CF3_C2H5F': ['CH3CF3(1)', 'C2H5F(187)', 'O2(2)'], 'CH3CF3_CH2FCH2F': ['CH3CF3(1)', 'S(187)', 'O2(2)'], 'CH3CF3_CH2FCHF2': ['CH3CF3(1)', 'S(187)', 'O2(2)'], 'CH3CF3_CH3CHF2': ['CH3CF3(1)', 'S(187)', 'O2(2)'], 'CH3CHF2_CH2F2': ['CH3CHF2(1)', 'CH2F2(39)', 'O2(2)'], 'CH3CHF2_CH2FCHF2': ['CH3CHF2(1)', 'S(179)', 'O2(2)'

In [15]:
mapping['C2H5F_CH2F2']

['C2H5F(1)', 'CH2F2(35)', 'O2(2)']

In [13]:
for blend in list_of_blends: 
    os.system(f'rm /work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts_species_in_core_redo/{blend}/input.py')


In [21]:
list_of_blends

['C2H5F_CH2F2',
 'C2H5F_CH2FCH2F',
 'C2H5F_CH2FCHF2',
 'C2H5F_CH3CHF2',
 'C2H5F_CH3F',
 'CH2F2_CH3CF3',
 'CH2F2_CH3F',
 'CH2FCH2F_CH2F2',
 'CH2FCH2F_CH2FCHF2',
 'CH2FCH2F_CH3CHF2',
 'CH2FCH2F_CH3F',
 'CH2FCHF2_CH2F2',
 'CH2FCHF2_CH3F',
 'CH3CF3_C2H5F',
 'CH3CF3_CH2FCH2F',
 'CH3CF3_CH2FCHF2',
 'CH3CF3_CH3CHF2',
 'CH3CHF2_CH2F2',
 'CH3CHF2_CH2FCHF2',
 'CH3F_CH3CF3',
 'CH3F_CH3CHF2']

In [27]:
list_ = list_of_blends



for blend in list_: 
    
    print(blend)
    
    dictionary_mapping = mapping[blend]
    print(dictionary_mapping)
    
    full_spc1_label, full_spc2_label = dictionary_mapping[0],dictionary_mapping[1]
    
    match1 = re.match('(\S+)\([0-9]+\)', full_spc1_label)   
    match2 = re.match('(\S+)\([0-9]+\)', full_spc2_label)   
    
    spc1, spc2 = match1.group(1), match2.group(1)

    
    print(f'Starting {blend}')
    
    #load the chemkin file of concatenated blends
    path_ = f'/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/concatenate_equal_parts/{blend}'
    species, reactions = load_chemkin_file(path_)
    
    #will save the species to a dictionary
    smiles = dict()
    
    spcs_we_dont_like = ['N2','Ne']
    #take their species
    for spc in species: 
        if spc.label not in spcs_we_dont_like:
            smiles[spc.label] = spc.smiles
        else:
            print(f'not adding: {spc.label}')
    
    assert('N2' not in smiles.keys())
    
    #remove duplicate values (smiles) in dictionary
    temp = []
    no_duplicates = dict()
    for key, val in smiles.items():
        if val not in temp:
            temp.append(val)
            no_duplicates[key] = val
            
    assert('N2' not in no_duplicates.keys())       
        
    input_file = write_rmg_input_file(no_duplicates, spc1, spc2)
    
    with open(f'/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts_species_in_core_redo/{blend}/input.py', 'w') as f: 
        f.write(input_file)
        
        
 

C2H5F_CH2F2
['C2H5F(1)', 'CH2F2(35)', 'O2(2)']
Starting C2H5F_CH2F2
not adding: N2
not adding: Ne
adding in species: C2H5F 
adding in species: O2 
adding in species: H2O 
adding in species: CH4 
adding in species: OH 
adding in species: H 
adding in species: O 
adding in species: H2 
adding in species: HO2 
adding in species: H2O2 
adding in species: CO 
adding in species: CO2 
adding in species: HCO 
adding in species: CH 
adding in species: CH2(T) 
adding in species: CH3 
adding in species: CH2O 
adding in species: HCCO 
adding in species: C2H 
adding in species: C2H2 
adding in species: H2CC 
adding in species: CH3O 
adding in species: CH2CO 
adding in species: C2H3 
adding in species: C2H4 
adding in species: C2H6 
adding in species: C2H5 
adding in species: CH2OH 
adding in species: CH3CO 
adding in species: F 
adding in species: HF 
adding in species: CH3F 
adding in species: CHF 
adding in species: CH2F2 
adding in species: CH2F 
adding in species: CHFO 
adding in species: CF 
a

In [26]:
match.group(1)

'C2H5F'